# Building Data Genome 2.0
## Breakout Detection
Biam! (pic.biam@gmail.com)

# Introduction

For breakout detection in the cleaned meters dataset will be used the [Breakout Detection package](https://github.com/twitter/BreakoutDetection) developed by Twitter. A brief introduction about this package can be found [here](https://blog.twitter.com/engineering/en_us/a/2014/breakout-detection-in-the-wild.html). Cleaned meter data (outliers and 24hs zero readings removed) are used for this analysis.

In [ ]:
library(BreakoutDetection)
library(tidyverse)

In [ ]:
# List of files in directory
files = list.files(path="..//data//meters//raw//", pattern="*.csv", full.names=TRUE, recursive=FALSE)
files

In [ ]:
# Apply method to all elementes
lapply(files, function(x){
  
  # meter name
  meter = str_split(str_split(x, "//")[[1]][5], ".csv")[[1]][1]
    
  # load data
  data = read_csv(x)
    
  # timestamp as time object
  data$timestamp = as.POSIXct(data$timestamp, format="%Y-%m-%d %H:%M:%OS")
    
  # Adds relative time (in hours) column
  data$RelativeTime = seq(0,length(data$timestamp)-1)
  
  # Creates df only with timestamp an Relative time
  df_res = data[,c(1, length(data))]
  
  # Loop over all columns (except timestamp and Relative time)
  for (i in seq(2,length(data)-1)){
    # Column name
    name = colnames(data)[i]
    # Creates df with only timestamp, relative time and selected building readings
    df = na.omit(data[, c(1,length(data),i)])
    # rename building column to work with package
    df = rename(df, count = 3)
    # Breakout detection
    res = breakout(df, min.size=168, method='multi', beta=.001, degree=1, plot=FALSE)
    # Creates dataframe with breakouts (only for merging)
    loc = data.frame("RelativeTime" = res$loc, "breakout" = res$loc)
    # Merge datasets
    df_res = merge(df_res, loc, by="RelativeTime", all.x = TRUE)
    # Rename column to building name
    df_res = rename(df_res, !!name := breakout)
    # Drop useless column
    df_res$count = NULL
    }
  
    # write file
    write_csv(df_res, paste0("..//data/meters//screening//breakout//", meter, "_breakout.csv"))
})